In [1]:
%env PLATFORM_API_URL=https://studio.dev.epistemix.cloud/v1

env: PLATFORM_API_URL=https://studio.dev.epistemix.cloud/v1


In [2]:
from epx import Job, ModelConfig, SynthPop

from pathlib import Path
import os

import pandas as pd
import plotly.express as px
import plotly.io as pio

# Use the Epistemix default plotly template
from epxexec.visual.utils import default_plotly_template
pio.templates["epistemix"] = default_plotly_template()
pio.templates.default = "epistemix"

os.environ["FRED_PROJECT"] = str(Path.cwd())
path_to_fred_project = os.environ["FRED_PROJECT"]

In [ ]:
mpox_config = ModelConfig(synth_pop=SynthPop("US_2010.v5", ["Dane_County_WI"]),
    start_date = "2022-06-01",
    end_date = "2024-06-01")

results_dir = "/home/epx/cl-results"

# Configure FRED job
mpox_job = Job(
    "model/main_monkeypox.fred",
    config=[mpox_config],
    key="mpox_job",
    results_dir=results_dir,
    size="hot",
    # Select FRED version compatible with selected model
    fred_version="10.1.1"
)

# Execute job
mpox_job.execute()

import time
# the following loop idles while we wait for the simulation job to finish
start = time.time()
timeout   = 3000 # timeout in seconds
idle_time = 3   # time to wait (in seconds) before checking status again
while str(mpox_job.status) != 'DONE':
    print(f"{mpox_job.status} at {time.time()-start}")
    if time.time() > start + timeout:
        msg = f"Job did not finish within {timeout / 60} minutes."
        raise RuntimeError(msg)
    time.sleep(idle_time)

str(mpox_job.status)

NOT STARTED at 0.0005114078521728516
NOT STARTED at 3.0040698051452637
NOT STARTED at 6.006180286407471
NOT STARTED at 9.009763956069946
NOT STARTED at 12.011634111404419
NOT STARTED at 15.015439510345459
NOT STARTED at 18.01799750328064
NOT STARTED at 21.021632432937622
NOT STARTED at 24.023305892944336
RUNNING at 27.03738808631897
RUNNING at 30.040996551513672
RUNNING at 33.04472255706787
RUNNING at 36.048287868499756
RUNNING at 39.051368713378906
RUNNING at 42.05386161804199
RUNNING at 45.05685520172119
RUNNING at 48.060423612594604
RUNNING at 51.06397533416748
RUNNING at 54.06732773780823
RUNNING at 57.07732677459717
RUNNING at 60.0804808139801
RUNNING at 63.08525490760803
RUNNING at 66.08733749389648
RUNNING at 69.09116268157959
RUNNING at 72.09475469589233
RUNNING at 75.09837317466736
RUNNING at 78.10196375846863
RUNNING at 81.10563516616821
RUNNING at 84.10740160942078
RUNNING at 87.11033082008362
RUNNING at 90.11403274536133
RUNNING at 93.11534190177917
RUNNING at 96.1190412044

In [ ]:
mpox_job.results.state(condition="BATHHOUSE", state="Visit", count_type="new")

In [ ]:
susceptible = mpox_job.results.state(
    condition="MPOX",
    state="Susceptible",
    count_type="new"
)

exposed = mpox_job.results.state(
    condition="MPOX",
    state="Exposed",
    count_type="new"
)

infectious = mpox_job.results.state(
    condition="MPOX",
    state="Infectious",
    count_type="new"
)


recovered = mpox_job.results.state(
    condition="MPOX",
    state="Recovered",
    count_type="new"
)

In [ ]:
susceptible

In [ ]:
susceptible.to_csv("sus_counts.csv")

In [ ]:
# rename the 'new' counts column to the name of the state
susceptible.rename(columns={'new':'Susceptible'},inplace=True)
exposed.rename(columns={'new':'Exposed'},inplace=True)
infectious.rename(columns={'new':'Infectious'},inplace=True)
recovered.rename(columns={'new':'Recovered'},inplace=True)

# build up the merged dataframe with counts for all states
all_states = pd.merge(susceptible,exposed)
all_states = pd.merge(all_states,infectious)
all_states = pd.merge(all_states,recovered)

#retrieve the sim_date from the column and add to the all_states frame.
all_states['sim_date']=mpox_job.get_job_date_table().sim_date

In [ ]:
all_states.to_csv("all_mpox_data_vals1.csv")

In [ ]:
all_states['Exposed_7day_avg'] = all_states.Exposed.rolling(7).mean()
all_states['Recovered_7day_avg'] = all_states.Recovered.rolling(7).mean()
all_states['Infectious_7day_avg'] = all_states.Infectious.rolling(7).mean()

all_state_7day_df = all_states[~all_states['Exposed_7day_avg'].isnull()]

fig = px.line(
    all_state_7day_df,
    x='sim_date',
    y=['Exposed_7day_avg','Infectious_7day_avg','Recovered_7day_avg'], # pass a list to plot all states],
    title="Mpox outbreak in County")
fig.update_layout( # customize font and legend orientation & position
    font_family="Epistemix Label",
    yaxis_title="New infections per day, 7 day average",
    xaxis_title="Date",
    legend_title="State",
    title_font_size=30
)
fig.show()

In [ ]:
mpox_job.delete(interactive=False)